# Sequence analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from plaid.utils import *
from plaid.evaluation import *
from tqdm.auto import tqdm, trange
import pandas as pd
from pathlib import Path
import torch
import os

# Methods
## PLAID

In [3]:
rootdir = Path("/data/lux70/plaid/artifacts/samples/by_length/")

In [4]:
device = torch.device("cuda")
rita_perplexity = RITAPerplexity(device=device)

/homefs/home/lux70/micromamba/envs/omegafold/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
length = str(32)

In [8]:
def sequence_properties(length, rootdir):
    length = str(length)

    sequences_dict = read_sequences_from_fasta(rootdir / f"{length}/generated/sequences.fasta")
    perplexities = []
    sequences = []
    
    for i in trange(64):
        sequence = sequences_dict[f"sequence_{i}"]
        ppl = rita_perplexity.calc_perplexity(sequence)
        perplexities.append(ppl)
        sequences.append(sequence)
    
    df = pd.DataFrame({
        "sequences": sequences,
        "perplexity": perplexities
    })
    
    df = calculate_df_protein_property_mp(df)
    return df

In [9]:
df = sequence_properties(32, rootdir) 

  0%|          | 0/64 [00:00<?, ?it/s]

In [16]:
df.hist(molecular_weight)

NameError: name 'molecular_weight' is not defined

# Natural

In [15]:
from evo.dataset import FastaDataset

pfam_path  = "/data/lux70/data/pfam/Pfam-A.fasta"
ds = FastaDataset(pfam_path)
dl = torch.utils.data.DataLoader(ds, batch_size=1024)
batch = next(iter(dl))

100%|██████████| 10.6G/10.6G [00:43<00:00, 264MB/s] 
100%|██████████| 10.6G/10.6G [01:07<00:00, 169MB/s]
